In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix


In [2]:
# 1. 数据加载
# 假设数据集是 CSV 格式
train_df = pd.read_csv(r"D:\Program Projects\Python Projects\temp\美版知乎问答\input\train.csv")  # 请根据实际文件路径修改
test1_df = pd.read_csv(r"D:\Program Projects\Python Projects\temp\美版知乎问答\input\test1.csv")    # 测试集（如果有）
submission = pd.read_csv(r"D:\Program Projects\Python Projects\temp\美版知乎问答\input\sample_submission.csv")    # 测试集（如果有）
test1_df = test1_df.drop_duplicates().reset_index(drop=True)

C:\Users\yx140\AppData\Local\Temp\ipykernel_88600\967476160.py:4: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  test1_df = pd.read_csv(r"D:\Program Projects\Python Projects\temp\美版知乎问答\input\test1.csv")    # 测试集（如果有）


In [3]:
test1_df = test1_df[~test1_df["test_id"].apply(lambda x: isinstance(x, str))]
test1_df = test1_df.drop_duplicates().reset_index(drop=True)
test1_df

,test_id,question1,question2
0,0,How does the Surface Pro himself 4 compare wit...,Why did Microsoft choose core m3 and not core ...
1,1,Should I have a hair transplant at age 24? How...,How much cost does hair transplant require?
2,2,What but is the best way to send money from Ch...,What you send money to China?
3,3,Which food not emulsifiers?,What foods fibre?
4,4,"How ""aberystwyth"" start reading?",How their can I start reading?
...,...,...,...
2345791,2345791,How do Peaks (TV series): Why did Leland kill ...,What is the most study scene in twin peaks?
2345792,2345792,"What does be ""in transit"" mean on FedEx tracking?",How question FedEx packages delivered?
2345793,2345793,What are some famous Romanian drinks (alcoholi...,Can a non-alcoholic restaurant be a huge success?
2345794,2345794,What were the best and worst things about publ...,What are the best and worst things examination...


In [4]:
print(train_df.columns)
print(test1_df.columns)

Index(['id', 'qid1', 'qid2', 'question1', 'question2', 'is_duplicate'], dtype='object')
Index(['test_id', 'question1', 'question2'], dtype='object')


In [5]:
train_df

,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0
...,...,...,...,...,...,...
404285,404285,433578,379845,How many keywords are there in the Racket prog...,How many keywords are there in PERL Programmin...,0
404286,404286,18840,155606,Do you believe there is life after death?,Is it true that there is life after death?,1
404287,404287,537928,537929,What is one coin?,What's this coin?,0
404288,404288,537930,537931,What is the approx annual cost of living while...,I am having little hairfall problem but I want...,0


In [6]:
# 2. 数据预处理
# 清理文本数据：这里可以添加更多的清理逻辑（如去除停用词、标点符号等）
def clean_text(text):
    if type(text)==str:
        text = text.lower()  # 转小写
        # 可以在这里加入额外的文本清理规则
        return text

train_df['question1'] = train_df['question1'].apply(clean_text)
train_df['question2'] = train_df['question2'].apply(clean_text)
train_df['question1'] = train_df['question1'].fillna("")
train_df['question2'] = train_df['question2'].fillna("")


test1_df['question1'] = test1_df['question1'].apply(clean_text)
test1_df['question2'] = test1_df['question2'].apply(clean_text)
test1_df['question1'] = test1_df['question1'].fillna("")
test1_df['question2'] = test1_df['question2'].fillna("")

In [7]:
import numpy as np 
def log_loss(y,pred): 
    return -np.mean(y * np.log(pred) + (1 - y) * np.log(1 - pred))

In [8]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss
import lightgbm as lgb
from scipy.sparse import hstack


# 3. 数据预处理：清理文本数据
def clean_text(text):
    if text is None:
        return ""
    text = text.lower()  # 转小写
    return text

train_df['question1'] = train_df['question1'].apply(clean_text)
train_df['question2'] = train_df['question2'].apply(clean_text)
test1_df['question1'] = test1_df['question1'].apply(clean_text)
test1_df['question2'] = test1_df['question2'].apply(clean_text)

# 4. 特征提取：使用 TF-IDF 向量化
vectorizer = TfidfVectorizer(max_features=10000, stop_words='english')

# 将两个问题拼接起来进行特征提取
X1 = vectorizer.fit_transform(train_df['question1'])
X2 = vectorizer.transform(train_df['question2'])

# 将两个问题的特征合并为一个矩阵
X = hstack([X1, X2])

# 目标变量
y = train_df['is_duplicate']

# 5. 训练模型：使用 LightGBM
# X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# 创建 LightGBM 模型
model = lgb.LGBMClassifier(max_depth=10, num_leaves=31, learning_rate=0.05, n_estimators=100)

# 训练模型
model.fit(X, y)

# 6. 模型评估：使用验证集评估模型
X1 = vectorizer.transform(test1_df['question1'])
X2 = vectorizer.transform(test1_df['question2'])
X = hstack([X1, X2])
y_pred = model.predict(X)  # 获取类别预测
y_pred_proba = model.predict_proba(X)[:, 1]  # 获取类别为1的概率预测

# 计算对数损失（Log Loss）
print(y_pred)


[LightGBM] [Info] Number of positive: 149263, number of negative: 255027
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 1.966596 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 489431
[LightGBM] [Info] Number of data points in the train set: 404290, number of used features: 18725
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.369198 -> initscore=-0.535660
[LightGBM] [Info] Start training from score -0.535660
[0 0 1 ... 0 0 0]


In [11]:
test1_df["is_duplicate"]=y_pred
test1_df

,test_id,question1,question2,is_duplicate
0,0,how does the surface pro himself 4 compare wit...,why did microsoft choose core m3 and not core ...,0
1,1,should i have a hair transplant at age 24? how...,how much cost does hair transplant require?,0
2,2,what but is the best way to send money from ch...,what you send money to china?,1
3,3,which food not emulsifiers?,what foods fibre?,0
4,4,"how ""aberystwyth"" start reading?",how their can i start reading?,0
...,...,...,...,...
2345791,2345791,how do peaks (tv series): why did leland kill ...,what is the most study scene in twin peaks?,0
2345792,2345792,"what does be ""in transit"" mean on fedex tracking?",how question fedex packages delivered?,0
2345793,2345793,what are some famous romanian drinks (alcoholi...,can a non-alcoholic restaurant be a huge success?,0
2345794,2345794,what were the best and worst things about publ...,what are the best and worst things examination...,0


In [13]:
test1_df=test1_df.drop(["question1","question2"],axis=1).drop_duplicates().reset_index(drop=True)
test1_df.to_csv(r'D:\Program Projects\Python Projects\temp\美版知乎问答\input\submission.csv',index=False)